## Use case
Auto-mpg data contains data of each car by years. With SM Feature Store, we can easily manage the features over years for each car. There are 56 cars that have records more than 1 year. We will create a feature store for the auto data, ingest the data of their first appearance, then update the feature store with new records by years. After the data is ingested, we show how to access data for training and inference purposes. And how to traverse over time to get feature point-in-time.

In [15]:
# SageMaker Python SDK version 2.x is required
import sagemaker
import sys

import boto3
import pandas as pd
import numpy as np
import io
import time
from time import gmtime, strftime, sleep
import datetime

region = 'us-west-2'
role = sagemaker.get_execution_role()
sess = sagemaker.Session()
bucket = sess.default_bucket()
prefix = 'sagemaker-studio-book/chapter05'

original_version = sagemaker.__version__
print(original_version)

2.49.1


In [114]:
print(int(round(time.time())))
print()

1630370909

Importing data from UCI

In [2]:
data_url='https://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
col_names=['mpg','cylinders', 'displacement', 'horsepower', 'weight', 
           'acceleration', 'model_year', 'origin', 'car_name']

df=pd.read_csv(data_url, delimiter='\s+', header=None, names=col_names, na_values='?')

In [46]:
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,car_name
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino


In [3]:
df.sort_values(by=['car_name', 'model_year'])

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,car_name
96,13.0,8,360.0,175.0,3821.0,11.0,73,1,amc ambassador brougham
9,15.0,8,390.0,190.0,3850.0,8.5,70,1,amc ambassador dpl
66,17.0,8,304.0,150.0,3672.0,11.5,72,1,amc ambassador sst
257,19.4,6,232.0,90.0,3210.0,17.2,78,1,amc concord
315,24.3,4,151.0,90.0,3003.0,20.1,80,1,amc concord
...,...,...,...,...,...,...,...,...,...
394,44.0,4,97.0,52.0,2130.0,24.6,82,2,vw pickup
197,29.0,4,90.0,70.0,1937.0,14.2,76,2,vw rabbit
309,41.5,4,98.0,76.0,2144.0,14.7,80,2,vw rabbit
325,44.3,4,90.0,48.0,2085.0,21.7,80,2,vw rabbit c (diesel)


In [4]:
df['car_name']=df['car_name'].astype('string')

In [25]:
import datetime
datetime.datetime(1970,1,1,8,0,0).timestamp()

28800.0

In [41]:
# data frames by years
d_df = {}
for yr in df['model_year'].unique():
    print(yr)
    d_df[str(yr)]=df[df['model_year']==yr]
    d_df[str(yr)]['event_time']=datetime.datetime(1900+yr, 1, 1, 8, 0, 0).timestamp()
#     print(d_df[str(yr)].shape)

70
(29, 10)
71
(28, 10)
72
(28, 10)
73
(40, 10)
74
(27, 10)
75
(30, 10)
76
(34, 10)
77
(28, 10)
78
(36, 10)
79
(29, 10)
80
(29, 10)
81
(29, 10)
82
(31, 10)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [42]:
d_df['70'].head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,car_name,event_time
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu,28800.0
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320,28800.0
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite,28800.0
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst,28800.0
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino,28800.0


## Create a feature group
We first start by creating feature group names for the auto-mpg data.

In [43]:
timestamp=strftime('%Y-%m-%d-%H-%M-%S', gmtime())

feature_group_name = 'auto-mpg-%s' % timestamp

In [44]:
from sagemaker.feature_store.feature_group import FeatureGroup

feature_group = FeatureGroup(
    name=feature_group_name, sagemaker_session=sess
)

In [45]:
record_identifier_feature_name = 'car_name'
event_time_feature_name = 'event_time'

In [46]:
feature_group.load_feature_definitions(data_frame=d_df['70'])

[FeatureDefinition(feature_name='mpg', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='cylinders', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>),
 FeatureDefinition(feature_name='displacement', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='horsepower', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='weight', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='acceleration', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>),
 FeatureDefinition(feature_name='model_year', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>),
 FeatureDefinition(feature_name='origin', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>),
 FeatureDefinition(feature_name='car_name', feature_type=<FeatureTypeEnum.STRING: 'String'>),
 FeatureDefinition(feature_name='event_time', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractiona

In [47]:
description='This feature group tracks the vehicle information such as mpg, and horsepower between 1970 and 1982.'
len(description)
# description has to be less than 128 characters

100

In [48]:
feature_group.create(
    s3_uri=f's3://{bucket}/{prefix}',
    enable_online_store=True,
    record_identifier_name=record_identifier_feature_name,
    event_time_feature_name=event_time_feature_name,
    description=description,
    role_arn=role
)

{'FeatureGroupArn': 'arn:aws:sagemaker:us-west-2:552106442228:feature-group/auto-mpg-2021-08-24-00-23-45',
 'ResponseMetadata': {'RequestId': '7be71ee4-2fc3-4e4b-8778-4cb5896d6fd5',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '7be71ee4-2fc3-4e4b-8778-4cb5896d6fd5',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '105',
   'date': 'Tue, 24 Aug 2021 00:23:57 GMT'},
  'RetryAttempts': 0}}

In [50]:
def check_feature_group_status(feature_group):
    status = feature_group.describe().get("FeatureGroupStatus")
    while status == "Creating":
        print("Waiting for Feature Group to be Created")
        time.sleep(5)
        status = feature_group.describe().get("FeatureGroupStatus")
    print(f"FeatureGroup {feature_group.name} successfully created.")

check_feature_group_status(feature_group)

Waiting for Feature Group to be Created
Waiting for Feature Group to be Created
Waiting for Feature Group to be Created
Waiting for Feature Group to be Created
Waiting for Feature Group to be Created
Waiting for Feature Group to be Created
Waiting for Feature Group to be Created
Waiting for Feature Group to be Created
Waiting for Feature Group to be Created
FeatureGroup auto-mpg-2021-08-24-00-23-45 successfully created.


In [33]:
# feature_group.delete()

## Ingest data into a feature group

In [51]:
for yr, df_auto in d_df.items():
    print(yr)
    print(df_auto.shape)
    feature_group.ingest(data_frame=df_auto, max_workers=1, max_processes = 1, wait=True)

70
(29, 10)
71
(28, 10)
72
(28, 10)
73
(40, 10)
74
(27, 10)
75
(30, 10)
76
(34, 10)
77
(28, 10)
78
(36, 10)
79
(29, 10)
80
(29, 10)
81
(29, 10)
82
(31, 10)


In [115]:
car_name = 'amc concord'
featurestore_runtime =  sess.boto_session.client(service_name='sagemaker-featurestore-runtime', 
                                                 region_name=region)
sample_record = featurestore_runtime.get_record(
    FeatureGroupName=feature_group_name, RecordIdentifierValueAsString=car_name
)

In [116]:
sample_record

{'ResponseMetadata': {'RequestId': '188e0ae6-a70b-4ebc-b116-dd4c810790ba',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '188e0ae6-a70b-4ebc-b116-dd4c810790ba',
   'content-type': 'application/json',
   'content-length': '527',
   'date': 'Tue, 31 Aug 2021 00:58:24 GMT'},
  'RetryAttempts': 0},
 'Record': [{'FeatureName': 'mpg', 'ValueAsString': '24.3'},
  {'FeatureName': 'cylinders', 'ValueAsString': '4'},
  {'FeatureName': 'displacement', 'ValueAsString': '151.0'},
  {'FeatureName': 'horsepower', 'ValueAsString': '90.0'},
  {'FeatureName': 'weight', 'ValueAsString': '3003.0'},
  {'FeatureName': 'acceleration', 'ValueAsString': '20.1'},
  {'FeatureName': 'model_year', 'ValueAsString': '80'},
  {'FeatureName': 'origin', 'ValueAsString': '1'},
  {'FeatureName': 'car_name', 'ValueAsString': 'amc concord'},
  {'FeatureName': 'event_time', 'ValueAsString': '315561600.0'}]}

In [ ]:
record = [{'FeatureName': 'mpg', 
           'ValueAsString': str(mpg)},
          {'FeatureName':'cylinders', 
           'ValueAsString': str(cylinders)},
          {'FeatureName':'displacement', 
           'ValueAsString': str(displacement)}, 
          {'FeatureName': 'horsepower', 
           'ValueAsString': str(horseposwer)},
          {'FeatureName': 'weight', 
           'ValueAsString': str(weight)},
          {'FeatureName': 'acceleration', 
           'ValueAsString': str(acceleration)},
          {'FeatureName': 'model_year', 
           'ValueAsString': str(model_year)},
          {'FeatureName': 'origin', 
           'ValueAsString': str(origin)},
          {'FeatureName': 'car_name', 
           'ValueAsString': str(car_name)},
          {'FeatureName': 'event_time', 
           'ValueAsString': str(int(round(time.time())))}]
featurestore_runtime.put_record(FeatureGroupName=feature_group_name, 
                                Record=record)

## Build training dataset

```python
def get_df_from_query(feature_group, query_string):
    query = feature_group.athena_query()
    
    print('Running ' + query_string)
    
    query.run(
        query_string=query_string,
        output_location=f's3://{bucket}/{prefix}/query_results/')
    query.wait()
    dataset = query.as_dataframe()
    return dataset
```

In [120]:
query = feature_group.athena_query()
table_name = query.table_name

query_string = ('SELECT * FROM "%s"' % table_name)
print('Running ' + query_string)

query.run(
        query_string=query_string,
        output_location=f's3://{bucket}/{prefix}/query_results/')
query.wait()
dataset = query.as_dataframe()

Running SELECT * FROM "auto-mpg-2021-08-24-00-23-45-1629764637"


In [121]:
dataset.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,car_name,event_time,write_time,api_invocation_time,is_deleted
0,17.5,8,305.0,145.0,3880.0,12.5,77,1,chevrolet caprice classic,220953600.0,2021-08-24 00:31:05.583,2021-08-24 00:24:59.000,False
1,26.0,4,97.0,75.0,2265.0,18.2,77,3,toyota corolla liftback,220953600.0,2021-08-24 00:31:05.583,2021-08-24 00:24:59.000,False
2,31.0,4,79.0,67.0,1950.0,19.0,74,3,datsun b210,126259200.0,2021-08-24 00:31:04.989,2021-08-24 00:24:56.000,False
3,26.0,4,98.0,79.0,2255.0,17.7,76,1,dodge colt,189331200.0,2021-08-24 00:31:04.603,2021-08-24 00:24:58.000,False
4,34.2,4,105.0,70.0,2200.0,13.2,79,1,plymouth horizon,284025600.0,2021-08-24 00:30:59.000,2021-08-24 00:25:02.000,False


In [123]:
df.shape

(398, 9)

In [122]:
dataset.shape

(398, 13)

In [141]:
query_string_2 = '''
SELECT * FROM "%s" WHERE model_year < 79
''' % table_name
print('Running ' + query_string_2)

query.run(
        query_string=query_string_2,
        output_location=f's3://{bucket}/{prefix}/query_results/')
query.wait()
dataset_2 = query.as_dataframe()

Running 
SELECT * FROM "auto-mpg-2021-08-24-00-23-45-1629764637" WHERE model_year < 79



In [130]:
query

AthenaQuery(catalog='AwsDataCatalog', database='sagemaker_featurestore', table_name='auto-mpg-2021-08-24-00-23-45-1629764637', sagemaker_session=<sagemaker.session.Session object at 0x7f2bbc351450>, _current_query_execution_id='78f69ce7-e492-42bc-a4ac-1eb4dcaf17e5', _result_bucket='sagemaker-us-west-2-552106442228', _result_file_prefix='sagemaker-studio-book/chapter05/query_results')

In [128]:
query

AthenaQuery(catalog='AwsDataCatalog', database='sagemaker_featurestore', table_name='auto-mpg-2021-08-24-00-23-45-1629764637', sagemaker_session=<sagemaker.session.Session object at 0x7f2bbc351450>, _current_query_execution_id='06b39bea-c2a0-4c96-bad5-f850f1d739f4', _result_bucket='sagemaker-us-west-2-552106442228', _result_file_prefix='sagemaker-studio-book/chapter05/query_results')

In [142]:
dataset_2.shape

(280, 13)

In [143]:
dataset_2.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,car_name,event_time,write_time,api_invocation_time,is_deleted
0,14.0,8,340.0,160.0,3609.0,8.0,70,1,plymouth 'cuda 340,28800.0,2021-08-24 00:31:00.268,2021-08-24 00:24:51.000,False
1,17.0,6,163.0,125.0,3140.0,13.6,78,2,volvo 264gl,252489600.0,2021-08-24 00:30:30.401,2021-08-24 00:25:00.000,False
2,19.0,6,225.0,95.0,3264.0,16.0,75,1,plymouth valiant custom,157795200.0,2021-08-24 00:31:05.584,2021-08-24 00:24:56.000,False
3,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino,28800.0,2021-08-24 00:31:04.984,2021-08-24 00:24:51.000,False
4,10.0,8,360.0,215.0,4615.0,14.0,70,1,ford f250,28800.0,2021-08-24 00:31:04.984,2021-08-24 00:24:51.000,False


In [131]:
query_string_3='''
SELECT *
FROM
    (SELECT *,
         row_number()
        OVER (PARTITION BY car_name
    ORDER BY  event_time desc, Api_Invocation_Time DESC, write_time DESC) AS row_number
    FROM "%s"
    where event_time < %.f)
WHERE row_number = 1 and
NOT is_deleted
''' % (table_name, datetime.datetime(1979, 1, 1, 8, 0, 0).timestamp())

print('Running ' + query_string_3)

query.run(
        query_string=query_string_3,
        output_location=f's3://{bucket}/{prefix}/query_results/')
query.wait()
dataset_3 = query.as_dataframe()

Running 
SELECT *
FROM
    (SELECT *,
         row_number()
        OVER (PARTITION BY car_name
    ORDER BY  event_time desc, Api_Invocation_Time DESC, write_time DESC) AS row_number
    FROM "auto-mpg-2021-08-24-00-23-45-1629764637"
    where event_time < 284025600)
WHERE row_number = 1 and
NOT is_deleted



In [132]:
dataset_3.shape

(212, 14)

In [144]:
dataset_2[dataset_2['car_name']=='amc gremlin']

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,car_name,event_time,write_time,api_invocation_time,is_deleted
5,21.0,6,199.0,90.0,2648.0,15.0,70,1,amc gremlin,28800.0,2021-08-24 00:31:06.532,2021-08-24 00:24:51.000,False
14,20.0,6,232.0,100.0,2914.0,16.0,75,1,amc gremlin,157795200.0,2021-08-24 00:31:06.538,2021-08-24 00:24:57.000,False
156,19.0,6,232.0,100.0,2634.0,13.0,71,1,amc gremlin,31564800.0,2021-08-24 00:31:06.532,2021-08-24 00:24:52.000,False
275,18.0,6,232.0,100.0,2789.0,15.0,73,1,amc gremlin,94723200.0,2021-08-24 00:31:06.538,2021-08-24 00:24:55.000,False


In [138]:
dataset_3[dataset_3['car_name']=='amc gremlin']

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,car_name,event_time,write_time,api_invocation_time,is_deleted,row_number
193,20.0,6,232.0,100.0,2914.0,16.0,75,1,amc gremlin,157795200.0,2021-08-24 00:31:06.538,2021-08-24 00:24:57.000,False,1


In [149]:
car_name = 'amc gremlin'
featurestore_runtime =  sess.boto_session.client(service_name='sagemaker-featurestore-runtime', 
                                                 region_name=region)
amc_gremlin = featurestore_runtime.get_record(
    FeatureGroupName=feature_group_name, RecordIdentifierValueAsString=car_name
)
amc_gremlin

{'ResponseMetadata': {'RequestId': 'a5cd0586-37d2-459f-93ac-a2cecaf40688',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'a5cd0586-37d2-459f-93ac-a2cecaf40688',
   'content-type': 'application/json',
   'content-length': '528',
   'date': 'Mon, 13 Sep 2021 18:23:37 GMT'},
  'RetryAttempts': 0},
 'Record': [{'FeatureName': 'mpg', 'ValueAsString': '20.0'},
  {'FeatureName': 'cylinders', 'ValueAsString': '6'},
  {'FeatureName': 'displacement', 'ValueAsString': '232.0'},
  {'FeatureName': 'horsepower', 'ValueAsString': '100.0'},
  {'FeatureName': 'weight', 'ValueAsString': '2914.0'},
  {'FeatureName': 'acceleration', 'ValueAsString': '16.0'},
  {'FeatureName': 'model_year', 'ValueAsString': '75'},
  {'FeatureName': 'origin', 'ValueAsString': '1'},
  {'FeatureName': 'car_name', 'ValueAsString': 'amc gremlin'},
  {'FeatureName': 'event_time', 'ValueAsString': '157795200.0'}]}

In [150]:
type(amc_gremlin)

dict

In [ ]:
query_string='''
SELECT *
FROM
    (SELECT *,
         row_number()
        OVER (PARTITION BY car_name
    ORDER BY  event_time desc, Api_Invocation_Time DESC, write_time DESC) AS row_number
    FROM "auto-mpg-2021-08-24-00-23-45-1629764637"
    where event_time <= %.f)
WHERE row_number = 1 and
NOT is_deleted
''' % datetime.datetime(1981, 1, 1, 8, 0, 0).timestamp()

df_query=get_df_from_query(feature_group, query_string)

In [67]:
dataset[dataset['car_name']=='amc concord']

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,car_name,event_time,write_time,api_invocation_time,is_deleted
249,24.3,4,151.0,90.0,3003.0,20.1,80,1,amc concord,315561600.0,2021-08-24 00:30:59.298,2021-08-24 00:25:03.000,False
310,19.4,6,232.0,90.0,3210.0,17.2,78,1,amc concord,252489600.0,2021-08-24 00:30:59.298,2021-08-24 00:25:00.000,False


In [107]:
query_string='''
SELECT *
FROM
    (SELECT *,
         row_number()
        OVER (PARTITION BY car_name
    ORDER BY  event_time desc, Api_Invocation_Time DESC, write_time DESC) AS row_number
    FROM "auto-mpg-2021-08-24-00-23-45-1629764637"
    where event_time <= %.f)
WHERE row_number = 1 and
NOT is_deleted
''' % datetime.datetime(1981, 1, 1, 8, 0, 0).timestamp()

df_query=get_df_from_query(feature_group, query_string)

Running 
SELECT *
FROM
    (SELECT *,
         row_number()
        OVER (PARTITION BY car_name
    ORDER BY  event_time desc, Api_Invocation_Time DESC, write_time DESC) AS row_number
    FROM "auto-mpg-2021-08-24-00-23-45-1629764637"
    where event_time <= 347184000)
WHERE row_number = 1 and
NOT is_deleted



In [108]:
df_query.shape

(278, 14)

In [109]:
df_query

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,car_name,event_time,write_time,api_invocation_time,is_deleted,row_number
0,26.5,4,140.0,72.0,2565.0,13.6,76,1,ford pinto,189331200.0,2021-08-24 00:31:05.406,2021-08-24 00:24:58.000,False,1
1,28.4,4,151.0,90.0,2670.0,16.0,79,1,buick skylark limited,284025600.0,2021-08-24 00:31:04.409,2021-08-24 00:25:02.000,False,1
2,30.5,4,97.0,78.0,2190.0,14.1,77,2,volkswagen dasher,220953600.0,2021-08-24 00:31:03.598,2021-08-24 00:24:59.000,False,1
3,18.5,8,360.0,150.0,3940.0,13.0,79,1,chrysler lebaron town @ country (sw),284025600.0,2021-08-24 00:31:04.102,2021-08-24 00:25:02.000,False,1
4,16.0,6,250.0,105.0,3897.0,18.5,75,1,chevroelt chevelle malibu,157795200.0,2021-08-24 00:31:05.284,2021-08-24 00:24:57.000,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,27.0,4,101.0,83.0,2202.0,15.3,76,2,renault 12tl,189331200.0,2021-08-24 00:31:04.011,2021-08-24 00:24:58.000,False,1
274,12.0,8,383.0,180.0,4955.0,11.5,71,1,dodge monaco (sw),31564800.0,2021-08-24 00:31:03.366,2021-08-24 00:24:52.000,False,1
275,27.0,4,97.0,88.0,2100.0,16.5,72,3,toyota corolla 1600 (sw),63100800.0,2021-08-24 00:31:04.074,2021-08-24 00:24:53.000,False,1
276,14.0,8,400.0,175.0,4464.0,11.5,71,1,pontiac catalina brougham,31564800.0,2021-08-24 00:31:04.080,2021-08-24 00:24:52.000,False,1


In [110]:
df_query[df_query['car_name']=='amc gremlin']

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,car_name,event_time,write_time,api_invocation_time,is_deleted,row_number
185,20.0,6,232.0,100.0,2914.0,16.0,75,1,amc gremlin,157795200.0,2021-08-24 00:31:06.538,2021-08-24 00:24:57.000,False,1


The query should be partitioned by `record_identifier_name` rather than `event_time_identifier_name`.